## Training Notebook

# Overview
    - Trained on kaggle GPU t4 x2


# Config and import

- Try: max length adjust


In [1]:
!pip install seqeval evaluate -q

In [2]:
from pathlib import Path
import os

import json
import argparse
from itertools import chain
from functools import partial

import torch
from transformers import AutoTokenizer, Trainer, TrainingArguments
from transformers import AutoModelForTokenClassification, DataCollatorForTokenClassification
import evaluate
from datasets import Dataset, features
import numpy as np
import pandas as pd

2024-03-05 23:45:49.283609: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-05 23:45:49.283732: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-05 23:45:49.475648: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
DATA_PATH = '../input/pii-detection-removal-from-educational-data'
TRAINING_MODEL_PATH = "microsoft/deberta-v3-base" #/kaggle/input/deberta_v3/keras/deberta_v3_base_en/2"
TRAINING_MAX_LENGTH = 1024
OUTPUT_DIR = "/kaggle/working/"

#print files with pathname
for dirname, _, filenames in os.walk(DATA_PATH):
    for filename in filenames:
        print(os.path.join(dirname, filename))

../input/pii-detection-removal-from-educational-data/sample_submission.csv
../input/pii-detection-removal-from-educational-data/train.json
../input/pii-detection-removal-from-educational-data/test.json


## Data Selection

In [4]:
#data from orginal training json
data = json.load(open(DATA_PATH+ "/train.json"))
org_data_df = pd.DataFrame(data)
base_df = org_data_df

print("Training Data: ", len(data))


#change base_df column labels to 'provided labels'
base_df = base_df.rename(columns={'labels': 'provided_labels'})

Training Data:  6807


In [5]:
#split data into training and validation set
def PIId2_split_by_sampling(df, test_ratio):
    """Split PII data into training and test set. 

    Args:
        df (pd.DataFrame): The input DataFrame.
        test_ratio (float, default=0.2): 
            The proportion of the dataset to include in the test split.

    Returns:
        train_df, test_df: Training and test splits of the input DataFrame.
    """
    # Get the number of rows in the DataFrame
    num_rows = len(df)
    
    # Get the split index
    split_idx = int(num_rows * test_ratio)
    
    # Shuffle the DataFrame rows
    df_shuffled = df.sample(frac=1, random_state=42)
    
    # Split the DataFrame into train and test
    train_df = df_shuffled.iloc[split_idx:]
    test_df = df_shuffled.iloc[:split_idx]
    
    return train_df, test_df

In [6]:
train_df, val_df = PIId2_split_by_sampling(base_df, .5)

In [7]:
print("train_df docs: ", train_df.shape[0])
print("val_df docs: ", val_df.shape[0])

target = [
    'B-NAME_STUDENT', 'B-EMAIL','B-USERNAME', 'B-ID_NUM', 'B-PHONE_NUM',
    'B-URL_PERSONAL', 'B-STREET_ADDRESS',
    'I-NAME_STUDENT', 'I-EMAIL','B-USERNAME', 'I-ID_NUM', 'I-PHONE_NUM',
    'I-URL_PERSONAL', 'I-STREET_ADDRESS'
]

#find labels missing in training and validation set
missing_base = set(target)- set(base_df['provided_labels'].explode().unique())
missing_train = set(target)- set(train_df['provided_labels'].explode().unique())
missing_val = set(target)- set(val_df['provided_labels'].explode().unique())

print("Missing labels in base dataset:", missing_base)
print("Missing labels in training dataset:", missing_train)
print("Missing labels in val dataset:", missing_val)

# If more labels are missing in training than test, swap the two dataframes
if len(missing_val) < len(missing_train):
    train_df, val_df = val_df, train_df
    print("Swapped train_df and val_df due to more missing labels in train_df.")
    
# Save the DataFrame to a JSON file
val_df.to_json(OUTPUT_DIR + "/val.json", orient="records", lines=True)

train_df docs:  3404
val_df docs:  3403
Missing labels in base dataset: {'I-EMAIL'}
Missing labels in training dataset: {'B-PHONE_NUM', 'I-ID_NUM', 'I-EMAIL', 'I-URL_PERSONAL', 'I-PHONE_NUM'}
Missing labels in val dataset: {'I-EMAIL'}
Swapped train_df and val_df due to more missing labels in train_df.


# Tokenize Data

In [8]:
#Set up labeling for NER with #Targets: B-Beginning entity, I-inside entity, O- outside entity

#Extract all unique labels w/ list comprehension. Use chain to flatten list of lists
all_labels = sorted(list(set(chain(*[x["labels"] for x in data]))))

#Create dictionary of label to id
label2id = {l: i for i,l in enumerate(all_labels)}

#Create dictionary of id to label
id2label = {v:k for k,v in label2id.items()}

#target labels identified in the training data- changed to all possible target labels
target = [
    'B-NAME_STUDENT', 'B-EMAIL','B-USERNAME', 'B-ID_NUM', 'B-PHONE_NUM',
    'B-URL_PERSONAL', 'B-STREET_ADDRESS',
    'I-NAME_STUDENT', 'I-EMAIL','B-USERNAME', 'I-ID_NUM', 'I-PHONE_NUM',
    'I-URL_PERSONAL', 'I-STREET_ADDRESS'
]

print(id2label)

{0: 'B-EMAIL', 1: 'B-ID_NUM', 2: 'B-NAME_STUDENT', 3: 'B-PHONE_NUM', 4: 'B-STREET_ADDRESS', 5: 'B-URL_PERSONAL', 6: 'B-USERNAME', 7: 'I-ID_NUM', 8: 'I-NAME_STUDENT', 9: 'I-PHONE_NUM', 10: 'I-STREET_ADDRESS', 11: 'I-URL_PERSONAL', 12: 'O'}


In [9]:
#prep data for NER training by tokenize the text and align labels to tokens
def tokenize(example, tokenizer, label2id, max_length):
    """This function ensures that the text is correctly tokenized and the labels 
    are correctly aligned with the tokens for NER training.

    Args:
        example (dict): The example containing the text and labels.
        tokenizer (Tokenizer): The tokenizer used to tokenize the text.
        label2id (dict): A dictionary mapping labels to their corresponding ids.
        max_length (int): The maximum length of the tokenized text.

    Returns:
        dict: The tokenized example with aligned labels.

    Reference: credit to https://www.kaggle.com/code/valentinwerner/915-deberta3base-training/notebook
    """

    # rebuild text from tokens
    text = []
    labels = []

    #iterate through tokens, labels, and trailing whitespace using zip to create tuple from three lists
    for t, l, ws in zip(
        example["tokens"], example["provided_labels"], example["trailing_whitespace"]
    ):
        text.append(t)
        
        #extend so we can add multiple elements to end of list if ws
        labels.extend([l] * len(t))
        if ws:
            text.append(" ")
            labels.append("O")

    #Tokenize text and return offsets for start and end character position. Limit length of tokenized text.
    tokenized = tokenizer("".join(text), return_offsets_mapping=True, max_length=max_length, truncation=True)

    #convert to np array for indexing
    labels = np.array(labels)

    # join text list into a single string 
    text = "".join(text)
    token_labels = []

    #iterate through each tolken
    for start_idx, end_idx in tokenized.offset_mapping:
        #if special tolken (CLS token) then append O
        #CLS : classification token added to the start of each sequence
        if start_idx == 0 and end_idx == 0:
            token_labels.append(label2id["O"])
            continue

        # case when token starts with whitespace
        if text[start_idx].isspace():
            start_idx += 1

        #append orginal label to token_labels
        token_labels.append(label2id[labels[start_idx]])

    length = len(tokenized.input_ids)

    return {**tokenized, "labels": token_labels, "length": length}

In [10]:
#load tokenizer based on pretrained model
tokenizer = AutoTokenizer.from_pretrained(TRAINING_MODEL_PATH)

#convert to hugging face Dataset object
ds = Dataset.from_pandas(train_df)

# Map the tokenize function to your dataset
ds = ds.map(
    tokenize,
    fn_kwargs={      # pass keyword args
        "tokenizer": tokenizer,
        "label2id": label2id,
        "max_length": TRAINING_MAX_LENGTH
    }, 
    num_proc=2   #apply in paralell using 3 processes
)

#todo check for UNK token due to fast tokenizer

tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/579 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:515: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


#0:   0%|          | 0/1702 [00:00<?, ?ex/s]

#1:   0%|          | 0/1701 [00:00<?, ?ex/s]

In [11]:
#TEMP check token and labels from first example before and after tokenzization 

#get first example from ds
x = ds[0]

# for before tokenization print all tokens that are not outside an entity
for t,l in zip(x["tokens"], x["provided_labels"]):
    if l != "O":
        print((t,l))

#print **** to seperate 
print("*"*10)

#print all tokens and label after tokenization
for t, l in zip(tokenizer.convert_ids_to_tokens(x["input_ids"]), x["labels"]):
    if id2label[l] != "O":
        print((t,id2label[l]))

**********


# Metrics and Training

In [12]:
from seqeval.metrics import recall_score, precision_score
from seqeval.metrics import classification_report
from seqeval.metrics import f1_score


def compute_metrics(p, all_labels):
    """Compute the F1, recall, precision metrics for a NER task.

    Args:
        p (Tuple[np.ndarray, np.ndarray]): The predictions and labels.
        all_labels (List[str]): The list of all possible labels.

    Returns:
        Dict[str, float]: The computed metrics (recall, precision, f1_score).
    Ref: https://www.kaggle.com/code/valentinwerner/915-deberta3base-training/notebook
    """
    #Note: seqeval framework for sequence labeling like NER
    
    # Unpack the predictions and labels
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [all_labels[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [all_labels[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    
    recall = recall_score(true_labels, true_predictions)
    precision = precision_score(true_labels, true_predictions)
    f5_score = (1 + 5*5) * recall * precision / (5*5*precision + recall)
    
    results = {
        'recall': recall,
        'precision': precision,
        'f5': f5_score
    }
    return results

In [13]:
#load
model = AutoModelForTokenClassification.from_pretrained(
    TRAINING_MODEL_PATH,        #pretrained model
    num_labels=len(all_labels), #num of unique labels for finetuning
    id2label=id2label,          #dicts for converting in fine tuning
    label2id=label2id,
    ignore_mismatched_sizes=True #pretrained model might have been trained on different num of labels
)

#collate list of sample from dataset into batches. 16 might be benefical for GPU architecture
collator = DataCollatorForTokenClassification(tokenizer, pad_to_multiple_of=16)

pytorch_model.bin:   0%|          | 0.00/371M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of DebertaV2ForTokenClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
#TODO - do we need evaluation set 

# final_ds = ds.train_test_split(test_size=0.2, seed=42) # cannot use stratify_by_column='group'
# final_ds

In [15]:
#Configure training process
#no validation set specified
training_args = TrainingArguments(
    output_dir= OUTPUT_DIR,  # Directory to save checkpoints and logs
    fp16 =True,               #mix-precision training on 16 bit to reduce memory and speed up training
    #learning_rate=2e-5,       # intial learning rate
    gradient_accumulation_steps=2,  #how many batches to acculumate gradient before optimization if batch size limited by GPU memory
    report_to="none",        #where training report progress, "none" prevents wandb login
    num_train_epochs=2,      # Number of training epochs
    per_device_train_batch_size=4,  # Batch size based per GPU
    #save_steps=500,          # Save model checkpoints every X steps
    do_eval = False,          #whether or not to perform eval during training
    evaluation_strategy="no",    # When to evaluate during training {no, steps or epoch}
    #eval_steps=100,          # Evaluate every X steps if stretegy is "steps"
    #logging_dir=OUTPUT_DIR+"/logs",    # Directory to save training logs
    logging_steps=100,       # Log training progress every X steps
    #load_best_model_at_end=True,   # Load the best model at the end of training
    metric_for_best_model="f5",  # Metric to determine the best model ("accuracy", f1...)
    #greater_is_better=True,      # if higher eval metric is better. True for f1 and acc
    save_total_limit=1,      # how many checkpoints to keep at end (1 means most recent)
    #lr_scheduler_type='cosine', #
    #warmup_ratio=0.1,           #steps to gradually increase learning rate. can help stabalize training at begining
    #weight_decay=0.01,          # l2 regularization to prevent overfitting
    
)

#inialize trainer for training and evaluation interface
trainer = Trainer(
    model=model, 
    args=training_args, 
    train_dataset=ds,
    data_collator=collator, 
    tokenizer=tokenizer,
    compute_metrics=partial(compute_metrics, all_labels=all_labels), #partial to fix all_label argument
)

In [16]:
%%time

#train model 
trainer.train()

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss
100,0.070700
200,0.003800
300,0.001800
400,0.001900


CPU times: user 22min 54s, sys: 8min 37s, total: 31min 32s
Wall time: 21min 57s


TrainOutput(global_step=426, training_loss=0.018410504240315286, metrics={'train_runtime': 1317.0808, 'train_samples_per_second': 5.167, 'train_steps_per_second': 0.323, 'total_flos': 3403520690477856.0, 'train_loss': 0.018410504240315286, 'epoch': 2.0})

In [17]:
trainer.save_model("deberta3base_pii2d_1024_v1")
tokenizer.save_pretrained("deberta3base_pii2d_1024_v1")

('deberta3base_pii2d_1024_v1/tokenizer_config.json',
 'deberta3base_pii2d_1024_v1/special_tokens_map.json',
 'deberta3base_pii2d_1024_v1/spm.model',
 'deberta3base_pii2d_1024_v1/added_tokens.json',
 'deberta3base_pii2d_1024_v1/tokenizer.json')